In [ ]:
# pip install geopandas rasterio pandas shapely pyproj

import pandas as pd
import geopandas as gpd
import rasterio
from rasterio.sample import sample_gen
from shapely.geometry import Point

# Input files
csv_file = r"CSV_file_with_lat_long.csv"
rainfall_tif = r"Rainfall_mm_per_day.tif"
temperature_tif = r"Temp_deg_C.tif"
output_file = r"Output.csv"

# Load CSV as GeoDataFrame
df = pd.read_csv(csv_file)

# Assuming CSV coords are in WGS84 (lon/lat) and in Decimal Degree
gdf = gpd.GeoDataFrame(
    df,
    geometry=[Point(xy) for xy in zip(df["Longitude"], df["Latitude"])], #The column names of Long and Lat
    crs="EPSG:4326"
)

print("✅ CSV loaded with", len(gdf), "points")
print("CSV CRS assumed:", gdf.crs)

# Function to sample raster values
def sample_raster(raster_path, gdf, column_name):
    with rasterio.open(raster_path) as src:
        print(f"\n--- Extracting from {raster_path} ---")
        print("Raster CRS:", src.crs)
        print("Raster bounds:", src.bounds)
        print("Raster nodata:", src.nodata)

        # Reproject points into raster CRS
        gdf_proj = gdf.to_crs(src.crs)

        # Generate coordinates for sampling
        coords = [(geom.x, geom.y) for geom in gdf_proj.geometry]

        # Sample raster at those points
        values = []
        for val in sample_gen(src, coords):
            v = val[0]
            if src.nodata is not None and v == src.nodata:
                v = None
            values.append(v)

        gdf[column_name] = values
        print(f"First 5 {column_name} values:", values[:5])
    return gdf

# Extract Rainfall & Temperature
gdf = sample_raster(rainfall_tif, gdf, "Rainfall")
gdf = sample_raster(temperature_tif, gdf, "Temperature")

# Save to CSV
gdf.drop(columns="geometry").to_csv(output_file, index=False)
print(f"\n✅ Extraction completed. File saved at: {output_file}")
